# Fresh Products

In [6]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import sys

# Move two levels up (to the project root) and append the `src` folder
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

# Append src to sys.path
sys.path.append(src_path)

from src import query_engines, dataframe_visualizer

q = query_engines.QueryEngines()

## Q1: Identify all kinds of segmentation

In this first part I want to segment all the possible partners. In Specialties, TP, NTP, MFCs and All groceries. We also check what is the perc of migrated Store Address Ids for each of them.

### What is the distribution of segments_lvl 1?

In [8]:
q.prepare_query(
    'backup/possible_segmentations.sql'
    ,params=None
    ,to_load_file='possible_segmentations'
    ,load_from_to_load_file='possible_segmentations'
)

possible_segmentations  = q.query_run_starburst()

grouped = possible_segmentations.groupby('segment_1').agg(
    distinct_sad_id = ('store_address_id','nunique'),
    migrated_sad_id = ('is_migrated', lambda x: x.sum())
).reset_index()

grouped['perc_m'] = grouped['migrated_sad_id']/grouped['distinct_sad_id']
grouped.sort_values(by='distinct_sad_id', ascending=False)

,segment_1,distinct_sad_id,migrated_sad_id,perc_m
2,Specialties,15387,1442,0.093715
3,Top Partner,10546,8496,0.805614
1,Non Top Partner,5850,1682,0.287521
0,MFC,256,245,0.957031


### What is the distribution of segment_lvl 2?

In [9]:
grouped = possible_segmentations.groupby('segment_2').agg(
    distinct_sad_id = ('store_address_id','nunique'),
    migrated_sad_id = ('is_migrated', lambda x: x.sum())
).reset_index()

grouped['perc_m'] = grouped['migrated_sad_id']/grouped['distinct_sad_id']
grouped.sort_values(by='distinct_sad_id', ascending=False)

,segment_2,distinct_sad_id,migrated_sad_id,perc_m
0,Groceries Partner,16396,10178,0.620761
2,Specialties,15387,1442,0.093715
1,MFC,256,245,0.957031


# Q2: Rest of metrics

In [18]:
START_DATE = "'2024-07-01'"
END_DATE = "'2024-09-15'"

params = [
    {'name':'start_date','value':START_DATE},
    {'name':'end_date','value':END_DATE}
]

q.prepare_query(
    'metrics.sql'
    ,params=params
    ,to_load_file='metrics'
    ,load_from_to_load_file=None
)

metrics  = q.query_run_starburst()
metrics.to_csv('metrics.csv')

/Users/jordicremades/Documents/repos/central-node/src/query_engines.py:138: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(self.tp__read_query, conn)


Open the following URL in browser for the external authentication:
https://starburst.g8s-data-platform-prod.glovoint.com/oauth2/token/initiate/e70ac55989bc2570ee8599905e8577e98ff58320a3c32b52915b93f397e61a42
